In [1]:
import pandas as pd 
import xarray as xr
import matplotlib.pyplot as plt 
import numpy as np
import os
import seaborn as sns

# import xycmap

import functions


In [2]:
tl_folder = f'/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/2024Nov_output/TL/'


In [4]:
%%time

# Define the list of landcover indices and percentiles
landcover_indices = [1, 2, 3, 4]
# percentiles = ['20percent', '30percent', '40percent']#, '50percent', '60percent']
percentiles = ['30percent']
method = 'once'  # Define your method here
column = 'Q100'  # Set the target column
location = 'langtang'

folder_langtang = '12a'
folder_mustang = '13a'


# Load elevation data and define the output path
elevation = pd.read_csv(f'/Users/varyabazilova/Desktop/paper2/downscaling_simple/coordinates_and_elevation_with_labels_{location}.csv')[['cellnr2', 'band_data']]
elevation = elevation.transpose()
elevation_list = elevation.loc['cellnr2'].tolist()  # Adjust if you need a specific list format


# Loop over each landcover index
for landcover_idx in landcover_indices:

    # result_df = pd.DataFrame()
    
    # Loop over each percentile
    for percentile in percentiles:
        print(f"Processing landcover index {landcover_idx} and percentile {percentile}")

        output_folder = f'/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/2024Nov_output/SL_{method}/output_{percentile}'
        # output_folder = f'/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/2024Nov_output/TL'
        # Define folder paths for each iteration
        # folder_path = f'/Volumes/Extreme SSD/202409_paper2_modelruns/30years/SL_{method}/{landcover_idx}landcover_{percentile}/{location}_climate_cut'
        folder_path = f'/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/SL_data_{method}/1landcover_{percentile}/{location}_climate_cut'
        # timestep workaround 
        df = functions.calculate_monthly_sediment_yield_all(pd.read_csv(folder_path + f'/{folder_langtang}/Sediment.out'), column)
        df = df[['month', 'year']]

        
        result_df = pd.DataFrame()

        # Iterate over files in the folder
        for folder_name in os.listdir(folder_path):
            # Skip unwanted files
            if folder_name.endswith('.csv') or folder_name.startswith('.'):
                continue

            file_path = os.path.join(folder_path, folder_name, 'Sediment.out')
            if os.path.isfile(file_path):
                # Read sediment data and calculate mean monthly value
                sediments = pd.read_csv(file_path)
                output_df = functions.calculate_monthly_sediment_yield_all(sediments, column)
                
                # Take the column and rename it
                name_column = output_df[column]
                column_name = f'{column}_{folder_name}'
                result_df[column_name] = name_column
                result_df.columns = [col[-3:] for col in result_df.columns]  # Rename columns for each folder


        
        # # Filter result_df by elevation list
        result_df = result_df[elevation_list]

        # Check for consistency with elevation list
        if elevation_list == result_df.columns.tolist():
            print("default land cover. monthly data: same")
            
            # Rename columns, merge with timestep, and add land cover info
            result_df.columns = elevation.loc['band_data']
            result_df = pd.concat([df, result_df], axis=1)  # Assuming `df` is defined elsewhere and aligns as expected
            result_df['land_cover'] = f'landcover{landcover_idx}'


            print(result_df)
            
            # Define the output file name and save the DataFrame to a CSV file
            output_filename = f'{location}_monthly_sum_elevation_{column}_landcover{landcover_idx}_{percentile}.csv'
            result_df.to_csv(os.path.join(output_folder, output_filename), index=False)



        
            print(f"Saved {output_filename}")
        else: print('its not the same AAAA!')

Processing landcover index 1 and percentile 30percent
default land cover. monthly data: same
     month  year         4485.0        3734.0  4880.0  4847.0        3908.0  \
0        9  1989       0.000000  0.000000e+00     0.0     0.0  0.000000e+00   
1       10  1989       0.000000  0.000000e+00     0.0     0.0  0.000000e+00   
2       11  1989       0.000000  0.000000e+00     0.0     0.0  0.000000e+00   
3       12  1989       0.000000  0.000000e+00     0.0     0.0  0.000000e+00   
4        1  1990       0.000000  0.000000e+00     0.0     0.0  0.000000e+00   
..     ...   ...            ...           ...     ...     ...           ...   
392      5  2022       0.000000  0.000000e+00     0.0     0.0  0.000000e+00   
393      6  2022  401182.400185  1.239064e+06     0.0     0.0  1.364211e+06   
394      7  2022  829515.500660  1.279493e+06     0.0     0.0  1.443771e+06   
395      8  2022  485112.433802  9.332687e+05     0.0     0.0  3.907959e+05   
396      9  2022       0.000000  4.691

FileNotFoundError: [Errno 2] No such file or directory: '/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/2024Nov_output/SL_once/output_30percent/langtang_monthly_sum_elevation_Q100_landcover1_30percent.csv'

In [ ]:
result_df.columns = elevation.loc['band_data']

In [ ]:
result_df

In [ ]:
%%time

# Define the list of landcover indices and percentiles
landcover_indices = [1, 2, 3, 4]
percentiles = ['20percent', '30percent', '40percent', '50percent', '60percent']
location = 'langtang'
method = 'daily' 

# Set the column and other parameters
column = 'Q100'

df = pd.DataFrame()  # Assuming `df` is initialized as needed

# Loop over each landcover index
for landcover_idx in landcover_indices:
    # Loop over each percentile
    for percentile in percentiles:
        print(f"Processing landcover index {landcover_idx} and percentile {percentile}")
        
        # Define folder paths
        folder_path = f'/Volumes/Extreme SSD/202409_paper2_modelruns/30years/SL_{method}/{landcover_idx}landcover_{percentile}/{location}_climate_cut'
        output_folder = f'/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/2024Nov_output/SL_{method}/output_{percentile}/'
        result_df = pd.DataFrame()

        # Iterate over files in the folder
        for folder_name in os.listdir(folder_path):
            # Skip hidden and .csv files
            if folder_name.endswith('.csv') or folder_name.startswith('.'):
                continue

            file_path = os.path.join(folder_path, folder_name, 'Sediment.out')
            if os.path.isfile(file_path):
                # Read sediment data and calculate monthly sediment yield
                sediments = pd.read_csv(file_path)
                output_df = functions.calculate_monthly_sediment_yield_all(sediments, column)
                
                # Take the specified column and rename it
                name_column = output_df[column]
                column_name = f'{column}_{folder_name}'
                result_df[column_name] = name_column
                result_df.columns = [col[-3:] for col in result_df.columns]

        # Filter and check columns with elevation list
        result_df = result_df[elevation_list]
        if elevation_list == result_df.columns.tolist():
            print("default land cover. monthly data: same")
        else:
            print("not the same")

        # Rename columns according to elevation, merge with timestep, and add land cover info
        result_df.columns = elevation.loc['band_data']
        result_df = pd.concat([df, result_df], axis=1)
        result_df['land_cover'] = f'landcover{landcover_idx}'

        # Save the DataFrame to a CSV file with both landcover and percentile in the filename
        output_filename = f'{location}_monthly_sum_elevation_{column}_landcover{landcover_idx}_{percentile}.csv'
        result_df.to_csv(os.path.join(output_folder, output_filename), index=False)

        print(f"Saved {output_filename}")


# test

In [ ]:
# test1 = pd.read_csv('/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/SL_data_once/1landcover_50percent/langtang_climate_cut/14d/Sediment.out')
# test2 = pd.read_csv('/Users/varyabazilova/Desktop/paper2/202406_modelruns/30years/SL_data_once/1landcover_60percent/langtang_climate_cut/14d/Sediment.out')

test1 = pd.read_csv('/Volumes/Extreme SSD/202409_paper2_modelruns/30years/SL_daily/1landcover_20percent/langtang_climate_cut/14d/Sediment.out')
test2 = pd.read_csv('/Volumes/Extreme SSD/202409_paper2_modelruns/30years/SL_daily/2landcover_40percent/langtang_climate_cut/14d/Sediment.out')
test3 = pd.read_csv('/Volumes/Extreme SSD/202409_paper2_modelruns/30years/SL_daily/2landcover_50percent/langtang_climate_cut/14d/Sediment.out')

In [ ]:
# test1

In [ ]:
test3.plot()

In [ ]:
test3.so.plot()
test3.Q100.plot()